In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
num_features = 3
num_neurons = 5
r = 0.06
batch_size = 16
num_epochs = 1000
# np.exp(r*t)*st*c_predict to be true value

In [10]:
m = tf.placeholder(tf.float32, [None, 1])
t = tf.placeholder(tf.float32, [None, 1])
c = tf.placeholder(tf.float32, [None, 1])
st = tf.placeholder(tf.float32, [None, 1])
w =  tf.Variable(tf.truncated_normal([1, num_neurons]), name="weights")
wm = tf.Variable(tf.truncated_normal([1, num_neurons]), name="weights_m")
bm = tf.Variable(tf.truncated_normal([num_neurons]), name="bias_m")
wt = tf.Variable(tf.truncated_normal([1, num_neurons]), name="weights_t")
bt = tf.Variable(tf.truncated_normal([num_neurons]), name="bias_t")

In [7]:
sigma_m = tf.nn.softplus(-1.0*tf.matmul(m, tf.exp(wm)) + bm, name="sigma_m")
sigma_t = tf.nn.sigmoid(tf.matmul(t, tf.exp(wt)) + bt, name="sigma_t")

In [11]:
c_predict = tf.reduce_sum(tf.multiply(tf.multiply(sigma_m, sigma_t), tf.exp(w)), name="c_predict")
c_predict_real = tf.multiply(tf.multiply(tf.exp(-1.0*r*t),st),c_predict, name="c_predict_real")
#loss = tf.losses.mean_squared_error(c, c_predict*np.exp(-1.0*r*t)*st)
loss = tf.losses.mean_squared_error(c_predict_real, c)
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss, "optimizer")

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("log/tf")
    sess.run(init)
    for _ in range(num_epochs):
        _, loss_value = sess.run([optimizer, loss], )